<h3>data/temperature/*.csv -> result/temperature/result.csv</h3>
<ol style="color: blue">
  <li>Remove empty lines </li>
  <li>Replace date to year </li>
  <li>Remove cells with older than 1990</li>
  <li>Groups by country+year and sum averages for temperatures</li>
</ol>

In [234]:
! cat data/temperature/*.csv | awk -f com/temperature/removeEmpty.awk | awk -f com/temperature/replaceYearDate.awk | awk -f com/temperature/filterYears.awk | awk -f com/temperature/groupByYearAverageSum.awk  > result/temperature/grouped.csv; awk -f com/countColumsRows.awk result/temperature/grouped.csv 

Columns: 4
Rows: 5785


<h3>data/energy/*.csv -> result/energy/result.csv</h3>
<ol style="color: blue">
  <li>Merge all countries </li>
  <li>Remove cells with older than 1990</li>
</ol>

In [319]:
! awk -f com/energy/merge.awk data/energy/*.csv | awk -f com/energy/filterYears.awk  > result/energy/merged.csv; awk -f com/countColumsRows.awk result/energy/merged.csv

Columns: 128
Rows: 7292


<h3>result/temperature/grouped.csv -> temperature_key.csv</h3>
<ol style="color: blue">
  <li>Makes temporary file which: Concates 4(country) And 1(year) columns and makes (key) column: key="country""year"</li>
</ol>

In [328]:
!cat result/temperature/grouped.csv | mlr --nidx --fs "," put -S '$key="key"; $key=NR == 1 ? "key" : $4.$1' > temperature_key.csv; awk -f com/countColumsRows.awk temperature_key.csv

Columns: 5
Rows: 5785


<h3>result/energy/merged.csv -> energy_key.csv</h3>
<ol style="color: blue">
  <li>Makes temporary file which: Concates 2(country) And 3(year) columns and makes (key) column: key="country""year"</li>
</ol>

In [329]:
!cat result/energy/merged.csv | mlr --nidx --fs "," put -S '$key="key"; $key=NR == 1 ? "key" : $2.$3' > energy_key.csv; awk -f com/countColumsRows.awk energy_key.csv 

Columns: 129
Rows: 7292


<h3>temperature_key.csv  energy_key.csv -> result/temperature_energy_joined.csv</h3>
<ol style="color: blue">
  <li>Join by key column</li>
  <li>Removes key column</li>
</ol>

In [330]:
! mlr --csv join -j key -f temperature_key.csv energy_key.csv | awk -F , 'BEGIN {OFS=FS}  {$1=""; sub(",", ""); print}' > result/temperature_energy_joined.csv; rm temperature_key.csv energy_key.csv; awk -f com/countColumsRows.awk result/temperature_energy_joined.csv

Columns: 132
Rows: 4460


<h3>Select columns and rows</h3>
OFS="[output separator]" { if ([column index] == "Russia") print [column index], ...}

In [375]:
!awk -F, 'OFS=","{if ($4 == "Russia") print $1, $2}' result/temperature_energy_joined.csv 

1990,-4.02292
1991,-4.237
1992,-4.90325
1993,-4.62175
1994,-4.89842
1995,-3.13017
1996,-4.81033
1997,-4.2965
1998,-5.28108
1999,-4.82975
2000,-4.54567
2001,-4.49067
2002,-4.12292
2003,-3.83283
2004,-4.49358
2005,-3.49808
2006,-4.763
2007,-2.97508
2008,-3.30542
2009,-4.6085
2010,-4.52058
2011,-3.40158
2012,-3.90175
2013,-2.26313


In [ ]:
!mlr --icsv --opprint cut -f country,year,coal_prod_change_pct result/temperature_energy_joined.csv

In [1]:
!awk -F, 'OFS=" "{if ($4 == "Russia") print $1, $2}' result/temperature_energy_joined.csv > result/russia_temperature.txt

In [9]:
!gnuplot -p -e 'filename="result/russia_temperature.txt"; fileout="visual/image.png"' visual/settings/settings_initial.g

In [7]:
!awk -F, 'OFS=" "{if ($1 == "1990") print $2, $3, $9, $14, $15, $4}' result/temperature_energy_joined.csv > result/temperature_1990.txt

In [21]:
!gnuplot -p -e 'filename="result/temperature_1990.txt"; fileout="visual/temperature_1990.png"' visual/settings/temperature_1990.g

In [18]:
!awk -F, 'OFS=" "{if ($1 == "2000") print $2, $3, $9, $14, $15, $4}' result/temperature_energy_joined.csv > result/temperature_2000.txt

In [26]:
!awk -F, 'OFS=" "{if ($1 == "2010") print $2, $3, $9, $14, $15, $4}' result/temperature_energy_joined.csv > result/temperature_2010.txt

In [41]:
!gnuplot -p -e 'filename1="result/temperature_2000.txt";filename2="result/temperature_2010.txt"; fileout="visual/temp_compare_2000vs2010.png"' visual/settings/compare_temperature_2000vs2010.g

In [42]:
!awk -F, 'OFS=" "{if ($1 == "1995") print $2, $3, $9, $14, $15, $4}' result/temperature_energy_joined.csv > result/temperature_1995.txt

In [58]:
!gnuplot -p -e 'filename1="result/temperature_2000.txt";filename2="result/temperature_2010.txt";filename3="result/temperature_1990.txt";filename4="result/temperature_1995.txt"; fileout="visual/compare_5_countries.png"' visual/settings/compare_5_countries.g

In [71]:
!gnuplot -p -e 'filename1="result/temperature_2000.txt";filename2="result/temperature_2010.txt";filename3="result/temperature_1990.txt";filename4="result/temperature_1995.txt"; fileout="visual/barplot.png"' visual/settings/barplot.g

In [124]:
!awk -F, 'OFS=" "{if ($2 <= "0" && $1 =="2001") print $2, $3, $31, $14, $15, $4}' result/temperature_energy_joined.csv > result/coal_cold_countries.txt

In [125]:
!cat result/coal_cold_countries.txt

-3.10008 0.237167 462.901 -1.991 -71.183 Canada
-17.0678 0.6745 0.0 0.245 0.576 Denmark
-17.5964 0.691167 0.0 -6.663 -0.198 Greenland
-4.49067 0.220417 1493.874 1.565 112.298 Russia


In [126]:
 !awk -F, 'OFS=" "{if ($2 >= "28" && $1 == "2001") print $2, $3, $31, $14, $15, $4}' result/temperature_energy_joined.csv > result/coal_hot_countries.txt

In [127]:
!cat result/coal_hot_countries.txt

28.6309 0.498583 0.0 3.471 0.153 Aruba
7.18892 0.217583 3.456 3.687 14.132 Austria
7.09225 0.196417 0.0 -2.519 -6.432 Belarus
28.7322 0.332583 0.0 11.122 0.474 Burkina Faso
9.81967 0.349667 2.953 0.672 2.06 Chile
7.59642 0.17425 8715.23 5.626 663.507 China
29.4769 0.686 0.0 -0.372 -0.013 Djibouti
5.973 0.227 0.0 4.072 2.198 Estonia
8.78708 0.168833 3203.836 1.087 263.854 Europe
28.4898 0.758333 0.0 8.93 0.113 Gambia
9.86175 0.355417 0.005 -7.615 -3.061 Georgia
9.09042 0.18475 677.528 1.766 70.195 Germany
6.81308 0.286083 398.239 12.583 46.355 Kazakhstan
4.781 0.3515 2.193 -5.114 -3.578 Kyrgyzstan
6.53925 0.226667 0.0 7.828 2.916 Latvia
7.12042 0.199333 0.0 16.832 13.04 Lithuania
29.3108 0.629417 0.0 1.18 0.052 Mali
28.8784 0.762083 0.0 4.376 0.221 Mauritania
28.118 0.69975 0.97 -2.078 -0.069 Niger
7.72708 0.245 171.613 -2.745 -7.883 North Korea
28.0833 0.44875 0.0 10.372 11.886 Oman
8.33383 0.24775 838.016 -0.709 -7.207 Poland
28.3057 0.568583 0.0 -3.896 -5.228 Qatar
9.63858 0.217917 6

In [128]:
!gnuplot -p -e 'filename1="result/coal_cold_countries.txt";filename2="result/coal_hot_countries.txt"; fileout="visual/coal_production.png"' visual/settings/coal_production.g

In [131]:
!awk -F, 'OFS=" "{if ($2 >= "28" && $1 == "2010") print $2, $3, $31, $14, $15, $4}' result/temperature_energy_joined.csv > result/coal_hot_countries_2010.txt

In [133]:
!awk -F, 'OFS=" "{if ($2 <= "0" && $1 == "2010") print $2, $3, $31, $14, $15, $4}' result/temperature_energy_joined.csv > result/coal_cold_countries_2010.txt

In [134]:
!gnuplot -p -e 'filename1="result/coal_cold_countries_2010.txt";filename2="result/coal_hot_countries_2010.txt"; fileout="visual/coal_production_2010.png"' visual/settings/coal_production.g

In [162]:
!awk -F, 'OFS=" "{if ($4 == "Germany") print $1, $31, $99}' result/temperature_energy_joined.csv > result/Germany.txt

In [210]:
!cat result/Germany.txt

1990 1454.182 576.293 150.246
1991 1234.531 566.701 153.953
1992 1135.491 501.247 156.16
1993 1021.692 481.207 155.404
1994 941.432 459.747 162.921
1995 919.346 461.357 168.117
1996 860.474 464.133 182.062
1997 824.191 436.496 178.967
1998 756.048 459.861 174.87
1999 735.945 428.751 186.783
2000 706.087 498.786 176.654
2001 677.528 505.24 178.334
2002 679.462 535.861 177.868
2003 672.2 562.052 185.157
2004 679.345 529.106 171.335
2005 657.931 520.359 165.427
2006 620.454 526.078 163.422
2007 632.836 517.691 149.697
2008 582.616 470.885 136.351
2009 539.803 431.347 127.462
2010 533.795 384.632 111.24
2011 543.483 400.19 104.655
2012 555.595 400.706 94.614
2013 524.756 403.03 86.041


In [166]:
!gnuplot -p -e 'filename="result/Germany.txt"; fileout="visual/oil_and_coal.png"' visual/settings/oil_and_coal.g

In [169]:
!awk -F, 'OFS=" "{if ($4 == "Turkey") print $1, $31, $99, $65}' result/temperature_energy_joined.csv > result/Turkey.txt

In [170]:
!cat result/Turkey.txt

1990 136.336 866.205 2.258
1991 127.976 1026.697 2.162
1992 140.924 966.702 2.109
1993 134.32 858.314 2.13
1994 140.803 800.279 2.119
1995 140.524 738.207 1.938
1996 142.998 725.3 2.194
1997 152.568 719.79 2.697
1998 162.179 680.775 6.017
1999 154.5 607.07 7.784
2000 145.238 534.404 6.805
2001 142.833 478.432 3.323
2002 132.12 458.409 4.025
2003 125.336 437.299 5.963
2004 122.49 406.147 7.327
2005 130.419 411.496 9.552
2006 153.051 392.315 9.637
2007 172.089 377.032 9.51
2008 193.919 378.875 10.841
2009 202.385 414.51 7.291
2010 203.804 427.6 7.266
2011 207.816 398.45 8.089
2012 197.908 384.654 6.734
2013 179.699 389.695 5.725


In [194]:
!gnuplot -p -e 'filename="result/Turkey.txt"; fileout="visual/gas_oil_coal_Turkey.png"' visual/settings/oil_gas_coal.g

In [208]:
!gnuplot -p -e 'filename="result/Turkey.txt"; fileout="visual/boxplot-turkey.png"' visual/settings/boxplot.g

In [209]:
!gnuplot -p -e 'filename="result/Germany.txt"; fileout="visual/boxplot-germany.png"' visual/settings/boxplot.g